In [36]:
import sys

sys.path.append("../")

import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from machine_learning.dataset_gen.station_dataframe_generator import StationDataframeGenerator
from window_generator import WindowGenerator

mpl.rcParams["figure.figsize"] = (8, 6)
mpl.rcParams["axes.grid"] = False

In [37]:
df = pd.read_feather("../../data/pollution/processed/interpolated/pm10/814.feather")
df.set_index("timestamp", inplace=True)

In [38]:
timestamp_s = df.index.map(pd.Timestamp.timestamp)

day = 24 * 60 * 60
year = (365.2425) * day

df["Day sin"] = np.sin(timestamp_s * (2 * np.pi / day))
df["Day cos"] = np.cos(timestamp_s * (2 * np.pi / day))
df["Year sin"] = np.sin(timestamp_s * (2 * np.pi / year))
df["Year cos"] = np.cos(timestamp_s * (2 * np.pi / year))
df.reset_index(inplace=True)



In [39]:
# Convert to radians.
wd_rad = df.pop("winddirection_10m") * np.pi / 180
df["winddirection_10m_sin"] = np.sin(wd_rad)
df["winddirection_10m_cos"] = np.cos(wd_rad)
df


,timestamp,temperature_2m,relativehumidity_2m,windspeed_10m,precipitation,pm10,Day sin,Day cos,Year sin,Year cos,winddirection_10m_sin,winddirection_10m_cos
0,2019-01-01 01:00:00,0.9,83,13.7,0.0,84.9443,2.588190e-01,0.965926,0.002738,0.999996,-0.996195,-0.087156
1,2019-01-01 02:00:00,0.6,82,14.8,0.0,59.5450,5.000000e-01,0.866025,0.003455,0.999994,-0.974370,-0.224951
2,2019-01-01 03:00:00,0.3,83,15.6,0.0,55.2941,7.071068e-01,0.707107,0.004172,0.999991,-0.945519,-0.325568
3,2019-01-01 04:00:00,0.3,85,15.2,0.0,39.3113,8.660254e-01,0.500000,0.004888,0.999988,-0.927184,-0.374607
4,2019-01-01 05:00:00,0.0,87,15.9,0.0,35.3945,9.659258e-01,0.258819,0.005605,0.999984,-0.838671,-0.544639
...,...,...,...,...,...,...,...,...,...,...,...,...
34400,2022-12-30 20:00:00,1.8,96,7.8,0.0,55.6152,-8.660254e-01,0.500000,-0.017532,0.999846,-0.224951,-0.974370
34401,2022-12-30 21:00:00,0.8,95,7.6,0.0,61.5726,-7.071068e-01,0.707107,-0.016815,0.999859,0.139173,-0.990268
34402,2022-12-30 22:00:00,0.2,95,7.9,0.0,72.9765,-5.000000e-01,0.866025,-0.016098,0.999870,0.406737,-0.913545
34403,2022-12-30 23:00:00,-0.1,94,9.9,0.0,74.3151,-2.588190e-01,0.965926,-0.015382,0.999882,0.544639,-0.838671


In [40]:
windowGenerator = WindowGenerator(input_width=24 * 14, label_width=24 * 14, shift=24 * 14, df=df.copy(), label_columns=["pm10"])

In [34]:
windowGenerator.plot(plot_col="winddirection_10m_sin")

In [35]:
windowGenerator.plot(plot_col="winddirection_10m_cos")

In [11]:
input, labels = windowGenerator.split_window(windowGenerator.windows_train)

In [12]:
labels.shape

TensorShape([2847, 168, 1])

In [9]:
windowGenerator.train.element_spec

(TensorSpec(shape=(None, 168, 10), dtype=tf.float64, name=None),
 TensorSpec(shape=(None, 168, 1), dtype=tf.float64, name=None))

In [10]:
for example_inputs, example_labels in windowGenerator.train.take(2):
  print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
  deb = example_inputs.numpy()
  print(f'Labels shape (batch, time, features): {example_labels.shape}')

Inputs shape (batch, time, features): (32, 168, 10)
Labels shape (batch, time, features): (32, 168, 1)
Inputs shape (batch, time, features): (32, 168, 10)
Labels shape (batch, time, features): (32, 168, 1)


2023-05-31 13:41:03.828695: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
